<a href="https://colab.research.google.com/github/suleiman-odeh/NLP_Project_Team16/blob/main/fine_tuning/fine_tuning_direct_Qwen2_5_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers peft trl bitsandbytes accelerate datasets

In [4]:
"""
Setup
"""

import torch
import gc
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

print("Ready to go.")

Ready to go.


In [ ]:
# Base model
model_name = "Qwen/Qwen2.5-7B"

# Define 4-Bit Configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load Base Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# fix pad tokens
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Prepare for LoRA
model = prepare_model_for_kbit_training(model)

# LoRA Config according to the paper
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type="CAUSAL_LM", # text generation model
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    )

print(f"Loaded BASE model: {model_name}")

In [ ]:
import pandas as pd
from datasets import Dataset

# Load dataset
filename = "QEvasion_cleaned.jsonl"

try:
    df = pd.read_json(filename, lines=True)
    print(f"Loaded {len(df)} rows using Pandas.")
except ValueError:
    # Fallback if it's a standard JSON list (not lines)
    df = pd.read_json(filename)
    print(f"Loaded {len(df)} rows using Pandas (Standard JSON).")

# Filter train and test sets
train_df = df[df['split_type'] == 'train']
# We keep the test set safe for later
test_df  = df[df['split_type'] == 'test']

print(f"   - Training pool: {len(train_df)}")
print(f"   - Test pool: {len(test_df)}")

# convert to hugging face dataset
full_train_dataset = Dataset.from_pandas(train_df)

# SPLIT (TRAIN vs VALIDATION)
# so it can match the paper
dataset_split = full_train_dataset.train_test_split(test_size=0.2175, seed=42)

train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

print(f"Final Setup:")
print(f"   - Training Samples: {len(train_dataset)}")
print(f"   - Validation Samples: {len(eval_dataset)}")

# ---------------------------------------------------------
# prompt based on the paper
# ---------------------------------------------------------

def formatting_prompts_func(examples):
    output_texts = []

    instruction = (
        "Based on a part of the interview where the interviewer asks a set of questions, "
        "classify the type of answer the interviewee provided for the following question "
        "into one of these categories:\n"
        "1. Clear Reply - The information requested is explicitly stated (in the requested form)\n"
        "2. Clear Non-Reply - The information requested is not given at all due to ignorance, need for clarification or declining to answer\n"
        "3. Ambivalent - The information requested is given in an incomplete way e.g. the answer is too general, partial, implicit, dodging or deflection."
    )

    # Define columns
    answers = examples['cleaned_answer']
    questions = examples['question']
    labels = examples['clarity_label']
    is_batch = isinstance(questions, list)

    if not is_batch:
        questions = [questions]
        answers = [answers]
        labels = [labels]

    for i in range(len(questions)):
        # Get the label text
        label_text = str(labels[i])

        # Construct Prompt
        text = f"{instruction}\n\n### Part of the interview ###\n{answers[i]}\n\n### Question ###\n{questions[i]}\n\nLabel: {label_text}"

        # Add EOS token
        text = text + tokenizer.eos_token
        output_texts.append(text)
    if not is_batch:
        return output_texts[0]

    return output_texts

# VERIFY
print("\n--- Sample Training Input ---")
print(formatting_prompts_func(train_dataset[:1])[0])

In [ ]:
"""
Training Loop
"""

from trl import SFTTrainer, SFTConfig

# sefine training arguments choice based on the settings used for OASST1 dataset, described in the paper
# QLORA: Efficient Finetuning of Quantized LLMs
training_args = SFTConfig(
    output_dir="./qwen-finetuned-evasion", # Where to save results

    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,         # Accumulate to simulate batch_size=16

    max_length=1024,
    gradient_checkpointing=True,           # Saves huge memory
    learning_rate=2e-4,                    # Standard QLoRA learning rate
    num_train_epochs= 1,
    lr_scheduler_type="constant",          # from the paper
    logging_steps=10,                      # Print stats every 10 steps
    eval_strategy="steps",           # Check Validation Set
    eval_steps=50,                         # Check validation every 50 steps
    save_strategy="no",                    # Don't save intermediate checkpoints
    fp16=False,                             # diffrent than the paper due to GPU constraints
    bf16=True,
    optim="paged_adamw_8bit",              # 8-bit Optimizer for our Colab GPU
    max_grad_norm=0.3,                     # from the paper
    adam_beta2=0.999,                      # from the paper
    report_to="none"                       # Prevents login popups
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    args=training_args,
)

# Start training
print("Starting Training...")
trainer.train()

# Save the adapter
trainer.model.save_pretrained("./final_adapter_qwen")
print("Training Complete. Adapter saved to './final_adapter_qwen'")

In [ ]:
"""
Inference: Check the testset
"""
from tqdm import tqdm

# Put model in evaluation mode
model.eval()

def generate_prediction(question, answer):
    instruction = (
        "Based on a part of the interview where the interviewer asks a set of questions, "
        "classify the type of answer the interviewee provided for the following question "
        "into one of these categories:\n"
        "1. Clear Reply - The information requested is explicitly stated (in the requested form)\n"
        "2. Clear Non-Reply - The information requested is not given at all due to ignorance, need for clarification or declining to answer\n"
        "3. Ambivalent - The information requested is given in an incomplete way e.g. the answer is too general, partial, implicit, dodging or deflection."
    )

    # Construct prompt without the label
    prompt = f"{instruction}\n\n### Part of the interview ###\n{answer}\n\n### Question ###\n{question}\n\nLabel:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode only the new tokens
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    prediction = full_text[len(prompt):].strip().split('\n')[0]
    return prediction

# Generate predictions for the test set
print(f"Generating predictions for {len(test_df)} samples...")
test_df['predicted_label'] = [
    generate_prediction(row['question'], row['cleaned_answer'])
    for _, row in tqdm(test_df.iterrows(), total=len(test_df))
]

# Display a few results
print("\n--- Sample Predictions ---")
print(test_df[['question', 'clarity_label', 'predicted_label']].head())

In [ ]:
"""
Evaluation
"""
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure labels are treated as strings for comparison
y_true = test_df['clarity_label'].astype(str).tolist()
y_pred = test_df['predicted_label'].astype(str).tolist()

# Calculate Accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Overall Accuracy: {accuracy:.4f}\n")

# Detailed Classification Report
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Confusion Matrix Visualization
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=sorted(set(y_true)),
            yticklabels=sorted(set(y_true)))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix: Qwen2.5 Evasion Classification')
plt.show()